### Week 6 Assignment- File Ingestion and Schema Validation

#### To get the size of the csv file.

In [3]:
import os
import time
#Size of the file
os.path.getsize('C:/Users/TALEHOUSE/Downloads/archive (1)/yelp_review.csv')

3791120545

#### Reading the data using Dask, Pandas, and Modin to compare computional efficiency.

#### Reading the data with Dask

In [4]:
from dask import dataframe as dd
start_time = time.time()
dask_df = dd.read_csv('C:/Users/TALEHOUSE/Downloads/archive (1)/yelp_review.csv')
end_time = time.time()
print("Time it takes to read csv with dask is: ",(end_time-start_time),"sec")

Time it takes to read csv with dask is:  0.025932788848876953 sec


#### Reading the data with Pandas.

In [6]:
import pandas as pd
start_time = time.time()
pandas_df = pd.read_csv('C:/Users/TALEHOUSE/Downloads/archive (1)/yelp_review.csv')
end_time = time.time()
print("Time it takes to read csv with pandas is: ",(end_time-start_time),"sec")

Time it takes to read csv with pandas is:  189.12285017967224 sec


#### Reading the data with Modin and Ray

In [7]:
import modin.pandas as mpd
import ray
ray.shutdown()
ray.init()
start_time = time.time()
modin_df = mpd.read_csv('C:/Users/TALEHOUSE/Downloads/archive (1)/yelp_review.csv')
end_time = time.time()
print("Time it takes to read csv with modin and ray is: ",(end_time-start_time),"sec")

2023-04-11 18:34:36,064	INFO worker.py:1553 -- Started a local Ray instance.


Time it takes to read csv with modin and ray is:  461.0015563964844 sec


#### RESULTS

Pandas DataFrame took around 189 seconds and Modin took 461 seconds whereas the same task is performed by Dask DataFrame took   0.02 seconds making it much less than a second time due to its impressive parallelization capabilities.    

Dask helps in doing data analysis faster because it parallelizes the existing frameworks like Pandas, Numpy, Scikit-Learn,   
and process data parallelly using the full potential of the machine’s CPU.
Hence Dask is better in reading large data sets than Pandas,Modin and Ray.

#### Reading the csv using the fastest method in handling large data set.

In [9]:
from dask import dataframe as dd
df = dask_df
df = dd.read_csv('C:/Users/TALEHOUSE/Downloads/archive (1)/yelp_review.csv',delimiter=',')

####  Data Exploration.

In [10]:
df.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 9 entries, review_id to cool
dtypes: object(5), int64(4)

In [11]:
df.columns

Index(['review_id', 'user_id', 'business_id', 'stars', 'date', 'text',
       'useful', 'funny', 'cool'],
      dtype='object')

In [12]:
#No, of Columns
len(df.columns)

9

#### Data Cleaning

In [13]:
# Removing special character
df.columns=df.columns.str.replace('[#,@,&]','')


In [14]:
#To remove white space from columns
df.columns = df.columns.str.replace(' ', '')
data=df.columns
data

Index(['review_id', 'user_id', 'business_id', 'stars', 'date', 'text',
       'useful', 'funny', 'cool'],
      dtype='object')

#### Validation: Creating a utility file

In [20]:
%%writefile utility.py
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.load(stream, Loader=yaml.Loader)
        except yaml.YAMLError as exc:
            logging.error(exc)

def col_header_val(df,table_config):
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

Overwriting utility.py


#### creating a yaml file

In [17]:
%%writefile store.yaml
file_type: csv
dataset_name: file
file_name: yelp_review
table_name: edsurv
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns:
    -review_id 
    -user_id 
    -business_id 
    -stars 
    -date 
    -text
    -useful 
    -funny
    -cool


Overwriting store.yaml


#### Reading Config file

In [ ]:
import utility as util
import yaml
config_data = util.read_config_file("store.yaml")

#### Inspecting the data of the Config file.

In [ ]:
config_data

#### Normal reading process of the file 

In [3]:
import pandas as pd
#df_sample = pandas_df
df_sample = pd.read_csv('C:/Users/TALEHOUSE/Downloads/archive (1)/yelp_review.csv',delimiter=',')

In [4]:
df_sample.head()

,review_id,user_id,business_id,stars,date,text,useful,funny,cool
0,vkVSCC7xljjrAI4UGfnKEQ,bv2nCi5Qv5vroFiqKGopiw,AEx2SYEUJmTxVVB18LlCwA,5,2016-05-28,Super simple place but amazing nonetheless. It...,0,0,0
1,n6QzIUObkYshz4dz2QRJTw,bv2nCi5Qv5vroFiqKGopiw,VR6GpWIda3SfvPC-lg9H3w,5,2016-05-28,Small unassuming place that changes their menu...,0,0,0
2,MV3CcKScW05u5LVfF6ok0g,bv2nCi5Qv5vroFiqKGopiw,CKC0-MOWMqoeWf6s-szl8g,5,2016-05-28,Lester's is located in a beautiful neighborhoo...,0,0,0
3,IXvOzsEMYtiJI0CARmj77Q,bv2nCi5Qv5vroFiqKGopiw,ACFtxLv8pGrrxMm6EgjreA,4,2016-05-28,Love coming here. Yes the place always needs t...,0,0,0
4,L_9BTb55X0GDtThi6GlZ6w,bv2nCi5Qv5vroFiqKGopiw,s2I_Ni76bjJNK9yG60iD-Q,4,2016-05-28,Had their chocolate almond croissant and it wa...,0,0,0


#### Reading the file using config file

In [ ]:
file_type = config_data['file_type']
source_file = "./" + config_data['file_name'] + f'.{file_type}'

#### Validating the header of the file

In [ ]:
util.col_header_val(df,config_data)

In [ ]:
print("columns of files are:" ,df.columns)
print("columns of YAML are:" ,config_data['columns'])

In [ ]:
if util.col_header_val(df,config_data)==0:
    print("validation failed")
else:
    print("col validation passed")

In [ ]:
import datetime
import csv
import gzip

from dask import dataframe as dd
df = dd.read_csv('C:/Users/TALEHOUSE/Downloads/archive (1)/yelp_review.csv',delimiter=',')

# Write csv in gz format in pipe separated text file (|)
df.to_csv('yelp_review.csv.gz',
          sep='|',
          header=True,
          index=False,
          quoting=csv.QUOTE_ALL,
          compression='gzip',
          quotechar='"',
          doublequote=True,
          line_terminator='\n')

#### Number of files in gz format folder

In [ ]:
import os
entries = os.listdir('C:/Users/TALEHOUSE/Downloads/archive (1)/yelp_review.csv.gz/')
for entry in entries:
    print(entry)

#### Size of the gz format folder

In [ ]:
os.path.getsize('C:/Users/TALEHOUSE/Downloads/archive (1)/yelp_review.csv.gz')